In [2]:
import cv2
import numpy as np
# Specify the paths for the 2 files 
protoFile = "pose_deploy_linevec_faster_4_stages.prototxt"
weightsFile = "pose_iter_160000.caffemodel"
# Read the network into Memory 
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
POSE_PAIRS = [
	[0,1],
	[1,2],
	[1,5],
	[2,3],
	[3,4],
	[5,6],
	[6,7],
	[1,14],
	[14,8],
	[14,11],
	[8,9],
	[9,10],
	[11,12],
	[12,13]
]

In [3]:

def getPosePrediction(image):
	# Specify the input image dimensions
	inWidth = 368
	inHeight = 368

	# Prepare the frame to be fed to the network 
	inpBlob = cv2.dnn.blobFromImage( 
		image, 1.0 / 255, (inWidth, inHeight), (0, 0, 0), swapRB=False, crop=False) 

	# Set the prepared object as the input blob of the network 
	net.setInput(inpBlob)
	return net.forward()

In [4]:
def overlayPrediction(output, frame):
	H = output.shape[2] 
	W = output.shape[3]
	threshold = 0.1
	height, width, _ = frame.shape
	# Empty list to store the detected keypoints 
	points = []
	for i in range(15): 
		# confidence map of corresponding body's part. 
		probMap = output[0, i, :, :] 

		# Find global maxima of the probMap. 
		minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

		# Scale the point to fit on the original image 
		x = (width * point[0]) / W 
		y = (height * point[1]) / H 

		if prob > threshold: 
			cv2.circle(frame, (int(x), int(y)), 15, (0, 255, 255), thickness=-1, lineType=cv2.FILLED) 
			cv2.putText(frame, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 0, 255), 3, lineType=cv2.LINE_AA) 
			points.append((int(x), int(y))) 
		else: 
			points.append(None)


	for pair in POSE_PAIRS: 
		partA = pair[0]
		partB = pair[1] 

		if points[partA] and points[partB]: 
			cv2.line(frame, points[partA], points[partB], (0, 255, 0), 3)
	return points

In [89]:
# Capturing the image we want to estimate pose on
# it is saved locally too
cam = cv2.VideoCapture(0)
image = None
while(True):
    result, image = cam.read()
    if result:
        cv2.imshow("pose-prediction", image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.imwrite("sample4.png", image)
        break
cam.release()
cv2.destroyAllWindows()

In [5]:
images = {}
# loop through num pictures, open and append
numPics = 4
for i in range(1, numPics + 1):
    fileName = "sample" + str(i) + ".png"
    images[fileName] = {}
    images[fileName]["rawImage"] = cv2.imread(fileName)


In [6]:
# Do prediction on each
def getImagePoints(image):
    imageCopy = image["rawImage"].copy()
    output = getPosePrediction(imageCopy)
    points = overlayPrediction(output, imageCopy)
    image["points"] = points
    image["overlay"] = imageCopy

In [8]:
# View prediction
def showImage(name, img):
    while(True):
        cv2.imshow(name, img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()

In [32]:
# Do eval on each

# get points from wrist, shoulder, and hip

#calculate angles needed
def calculateAngle(anglePoints):
    firstPoint = anglePoints[0]
    secondPoint = anglePoints[1]
    thirdPoint = anglePoints[2]
    
    ba = firstPoint - secondPoint
    bc = thirdPoint - secondPoint
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(cosine_angle))

#get the points needed for angles
def calculateImageAngles(image):
    angles = {
        "left_hip_shoulder_wrist": { "pointsIdx": [11,5,7] },
        "right_hip_shoulder_wrist": { "pointsIdx": [8,2,4] },
        "left_shoulder_hip_vertical": { "pointsIdx": [5,11], "extendPoint": 11 },
        "right_shoulder_hip_vertical": { "pointsIdx": [2,8], "extendPoint": 8 }
    }
    points = image["points"]
    for angle in angles:
        pIdxs = angles[angle]["pointsIdx"]
        anglePoints = []
        for pIdx in pIdxs:
            if points[pIdx] != None:
                anglePoints.append(np.array(points[pIdx]))

        if "extendPoint" in angles[angle]:
            extPointIdx = angles[angle]["extendPoint"]
            if points[extPointIdx] != None:
                extPoint = list(points[extPointIdx][:])
                extPoint[1] += 30
                anglePoints.append(np.array(extPoint))
        
        if len(anglePoints) != 3:
            angles[angle]["calculated"] = None
        else:
            result = calculateAngle(anglePoints)
            angles[angle]["calculated"] = result
    image["angles"] = angles

In [30]:
#define angle ranges as optimal, okay, or suboptimal
def getLeastOptimal(arg1, arg2, t):
    if arg1 and arg2:
        if t == "L":
            return max([arg1, arg2])
        elif t == "S":
            return min([arg1, arg2])
    elif arg1:
        return arg1
    return arg2

def defineErgonomics(image):
    leftWS = image["angles"]["left_hip_shoulder_wrist"]["calculated"]
    rightWS = image["angles"]["right_hip_shoulder_wrist"]["calculated"]
    leftHS = image["angles"]["left_shoulder_hip_vertical"]["calculated"]
    rightHS = image["angles"]["right_shoulder_hip_vertical"]["calculated"]

    WSCalc = getLeastOptimal(leftWS, rightWS, "L")
    HSCalc = getLeastOptimal(leftHS, rightHS, "S")

    WS = ""
    if WSCalc == None or WSCalc > 90:
        WS = "okay"
    else:
        WS = "optimal"

    HS = ""
    if HSCalc == None or HSCalc < 120:
        HS = "suboptimal"
    elif 120 <= HSCalc <= 160:
        HS = "okay"
    else:
        HS = "optimal"

    #categorize combination of movements into best, okay, or worst heights
    score = ""
    if HS == "suboptimal":
        score = "worst ergonomics"
    elif HS == "optimal" and WS == "optimal":
        score = "best ergonomics"
    else: 
        score = "okay ergonomics"
    
    image["ergonomics"] = {"score": score, "WSscore": WS, "HSscore": HS }

In [31]:
#  Test doing all calculations on all images
for fileName, image in images.items():
    getImagePoints(image)
    calculateImageAngles(image)
    defineErgonomics(image)

#print(images["sample1.png"]["points"])
#print(images["sample1.png"]["angles"])
print(images["sample1.png"]["ergonomics"]["score"])
#showImage("overlay", images["sample1.png"]["overlay"])

Not enough points to calculate angle
Not enough points to calculate angle
best ergonomics


In [39]:
for image in images:
    score = images[image]["ergonomics"]["score"]
    hs = images[image]["ergonomics"]["HSscore"]
    ws = images[image]["ergonomics"]["WSscore"]
    print(image, "Hip Angle", hs, ",  Wrist Angle", ws, ", score", score)

sample1.png Hip Angle optimal Wrist Angle optimal score best ergonomics
sample2.png Hip Angle optimal Wrist Angle okay score okay ergonomics
sample3.png Hip Angle optimal Wrist Angle optimal score best ergonomics
sample4.png Hip Angle okay Wrist Angle optimal score okay ergonomics
